In [1]:
!pip install geopy
!pip install folium
print("Installed!")

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Installed!


In [2]:
#call Foursquare API

CLIENT_ID = 'CSMOKQMDXCIFEIPPDABTJK4FBBQGXCPJMCVE2NDNJMCZUAR0' # Put Your Client Id
CLIENT_SECRET = '1GWQQ1RKJ4WKSXS4CV5BSF14VMZIBPEHI4ORLMJQSJ0GLZGH' # Put You Client Secret 
VERSION = '20180604'
LIMIT = 100
radius= 5000
print('Your credentials:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentials:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


In [3]:
#install libraries

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.options.display.width = 0
from sklearn.cluster import KMeans
import sklearn.cluster.k_means_
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Libraries imported.')

Libraries imported.


In [4]:
#load csv with cities to be examined and create new dataframe
df = pd.read_csv (r'UK and Spain.csv')
print (df)

                          city      lat      lng         country  population
0                       London  51.5000  -0.1167  United Kingdom     8567000
1                       Madrid  40.4000  -3.6834           Spain     5567000
2                    Barcelona  41.3833   2.1834           Spain     4920000
3                   Birmingham  52.4750  -1.9200  United Kingdom     2285000
4                   Manchester  53.5004  -2.2480  United Kingdom     2230000
5                        Leeds  53.8300  -1.5800  United Kingdom     1529000
6                    Sheffield  53.3667  -1.5000  United Kingdom     1292900
7                      Sevilla  37.4050  -5.9800           Spain     1212045
8                      Glasgow  55.8744  -4.2507  United Kingdom     1160000
9          Newcastle upon Tyne  55.0004  -1.6000  United Kingdom      882000
10                      Bilbao  43.2500  -2.9300           Spain      875552
11                    Caerdydd  51.5000  -3.2250  United Kingdom      861400

In [5]:
#print the number of cities
print('Number of cities:',df.shape[0])

Number of cities: 94


In [6]:
df['country'].value_counts()

United Kingdom    51
Spain             43
Name: country, dtype: int64

In [7]:
#print the data types included in the dataframe
df.dtypes

city           object
lat           float64
lng           float64
country        object
population      int64
dtype: object

In [8]:
#plot the cities of the dataframe in the form of a map
map_world = folium.Map()

# add markers to map
for lat, lng, city in zip(df['lat'], df['lng'], df['city']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_world)  
    
map_world

In [9]:
#call Foursquare API through function that returns list of venues 
def getVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    venues.columns = ['city', 'lat', 'lng', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(venues)

In [10]:
# Use function that returns list of venues to assign venues for each city of the initial dataframe

venues = getVenues(names=df['city'], latitudes=df['lat'],longitudes=df['lng'])

London
Madrid
Barcelona
Birmingham
Manchester
Leeds
Sheffield
Sevilla
Glasgow
Newcastle upon Tyne
Bilbao
Caerdydd
Nottingham
Liverpool
Valencia
Zaragoza
Southend-on-Sea
Bristol
Málaga
Edinburgh
Brighton
Bradford
Leicester
Sunderland
Belfast
Portsmouth
Bournemouth
Middlesbrough
Murcia
Stoke-on-Trent
Granada
Coventry
Southampton
Vigo
Las Palmas de Gran Canaria
Palma
Reading
Donostia
Santa Cruz de Tenerife
Gijón
Valladolid
Córdoba
Alicante
Kingston upon Hull
Pamplona
Blackpool
Plymouth
Luton
Oviedo
Vitoria-Gasteiz
Cartagena
Oxford
Norwich
Aberdeen
Marbella
Mataró
Castellón de la Plana
Almería
Burgos
Salamanca
York
Albacete
Dundee
Huelva
Ipswich
Logroño
Ciudad de Melilla
Peterborough
Badajoz
León
Cambridge
Tarragona
Ourense
Jaén
Exeter
Algeciras
Bath
Santiago de Compostela
Chester
Derry
Ciudad de Ceuta
Toledo
Carlisle
Scarborough
Arrecife
Mérida
Inverness
Perth
Dover
Dumfries
Omagh
Fort William
Kirkwall
Lerwick


In [11]:
#print first rows on the new dataframe that includes cities, venues and their coordinates
print(venues.shape)
venues.head()

(7874, 7)


,city,lat,lng,Venue,Venue Latitude,Venue Longitude,Venue Category
0,London,51.5,-0.1167,The London Eye,51.503287,-0.119594,Scenic Lookout
1,London,51.5,-0.1167,Elizabeth Tower (Big Ben) (Big Ben (Elizabeth ...,51.500620,-0.124578,Monument / Landmark
2,London,51.5,-0.1167,The Old Vic,51.502176,-0.109686,Theater
3,London,51.5,-0.1167,Southbank Centre,51.505843,-0.116985,Performing Arts Venue
4,London,51.5,-0.1167,National Theatre,51.507376,-0.114793,Theater


In [12]:
#count the number of venues by city

venues.groupby('city').count()

,lat,lng,Venue,Venue Latitude,Venue Longitude,Venue Category
city,,,,,,
Aberdeen,93,93,93,93,93,93
Albacete,50,50,50,50,50,50
Algeciras,68,68,68,68,68,68
Alicante,69,69,69,69,69,69
Almería,100,100,100,100,100,100
Arrecife,86,86,86,86,86,86
Badajoz,82,82,82,82,82,82
Barcelona,100,100,100,100,100,100
Bath,66,66,66,66,66,66


In [13]:
# one hot encoding
eu_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
eu_onehot['city'] = venues['city'] 
eu_onehot.head()

,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Internet Cafe,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monastery,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Paella Restaurant,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Pub,Public Art,Racecourse,Racetrack,Record Shop,Recreation Center,Rental Car Location,Reservoir,Resort,Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Rugby Pitch,Rugby Stadium,Russian Restaurant,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Scottish Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South A

In [14]:
#confirm new size
eu_onehot.shape

(7874, 347)

In [15]:
#group rows by city and by taking the mean of the frequency of occurrence of each category
groupedvenues = eu_onehot.groupby('city').mean().reset_index()
groupedvenues

,city,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Ballroom,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Internet Cafe,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monastery,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Outlet Store,Paella Restaurant,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Café,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Portuguese Restaurant,Pub,Public Art,Racecourse,Racetrack,Record Shop,Recreation Center,Rental Car Location,Reservoir,Resort,Rest Area,Restaurant,River,Road,Rock Club,Roof Deck,Rugby Pitch,Rugby Stadium,Russian Restaurant,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Scottish Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,So

In [16]:
#confirm new size
groupedvenues.shape

(94, 347)

In [17]:
#print each city with the top 5 most common venues
num_top_venues = 5

for cityname in groupedvenues['city']:
    print("----"+cityname+"----")
    temp = groupedvenues[groupedvenues['city'] == cityname].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aberdeen----
           venue  freq
0            Bar  0.10
1  Grocery Store  0.08
2     Restaurant  0.06
3           Café  0.06
4       Beer Bar  0.05


----Albacete----
                venue  freq
0  Spanish Restaurant  0.06
1    Tapas Restaurant  0.06
2               Hotel  0.06
3                Café  0.06
4      Ice Cream Shop  0.04


----Algeciras----
              venue  freq
0  Tapas Restaurant  0.12
1             Hotel  0.10
2               Pub  0.07
3       Coffee Shop  0.07
4    Ice Cream Shop  0.04


----Alicante----
                venue  freq
0    Tapas Restaurant  0.12
1  Spanish Restaurant  0.07
2               Plaza  0.07
3          Restaurant  0.06
4      Breakfast Spot  0.04


----Almería----
                venue  freq
0    Tapas Restaurant  0.18
1  Spanish Restaurant  0.07
2                 Pub  0.05
3                Café  0.05
4          Restaurant  0.05


----Arrecife----
                venue  freq
0          Restaurant  0.08
1               Hotel  0.08
2  Spa

In [18]:
#write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
#create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['city'] = groupedvenues['city']

for ind in np.arange(groupedvenues.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(groupedvenues.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aberdeen,Bar,Grocery Store,Café,Restaurant,Hotel,Park,Supermarket,Beer Bar,Sandwich Place,Seafood Restaurant
1,Albacete,Hotel,Tapas Restaurant,Spanish Restaurant,Café,Bar,Ice Cream Shop,Plaza,Park,Breakfast Spot,Brewery
2,Algeciras,Tapas Restaurant,Hotel,Pub,Coffee Shop,Ice Cream Shop,Spanish Restaurant,Harbor / Marina,Beach,Restaurant,Chinese Restaurant
3,Alicante,Tapas Restaurant,Spanish Restaurant,Plaza,Restaurant,Beach,Breakfast Spot,Pizza Place,Ice Cream Shop,Gym,Supermarket
4,Almería,Tapas Restaurant,Spanish Restaurant,Café,Pub,Restaurant,Mediterranean Restaurant,Beach,Italian Restaurant,Lounge,Bar


In [20]:
# set number of clusters
kclusters = 8
groupedvenues_clustering = groupedvenues.drop('city', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(groupedvenues_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[0 1 3 1 1 3 1 3 0 0]
94


In [21]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
cities_merged = df

#join dataframes
cities_merged = cities_merged.join(city_venues_sorted.set_index('city'), on='city')
cities_merged.head()

,city,lat,lng,country,population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,London,51.5000,-0.1167,United Kingdom,8567000,3,Hotel,Theater,Cocktail Bar,Clothing Store,Grocery Store,History Museum,Park,Art Museum,Garden,Wine Bar
1,Madrid,40.4000,-3.6834,Spain,5567000,1,Plaza,Tapas Restaurant,Park,Restaurant,Art Museum,Art Gallery,Museum,Hotel,Coffee Shop,Café
2,Barcelona,41.3833,2.1834,Spain,4920000,3,Hotel,Park,Plaza,Pizza Place,Café,Dessert Shop,Bakery,Historic Site,Tapas Restaurant,Gastropub
3,Birmingham,52.4750,-1.9200,United Kingdom,2285000,5,Indian Restaurant,Pub,Coffee Shop,Bar,Hotel,Fast Food Restaurant,Restaurant,Middle Eastern Restaurant,Park,Concert Hall
4,Manchester,53.5004,-2.2480,United Kingdom,2230000,5,Coffee Shop,Pub,Park,Bar,Hotel,Beer Bar,Pizza Place,Café,Sandwich Place,Vegetarian / Vegan Restaurant


In [22]:
# create map
map_clusters = folium.Map()

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cities_merged['lat'], cities_merged['lng'], cities_merged['city'], cities_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [23]:
#count cities per clusters

i=0
for i in range(0,8):
    print('K-means for {} has {} cities' .format(i,cities_merged.loc[cities_merged['Cluster Labels'] == i].shape[0]))

K-means for 0 has 17 cities
K-means for 1 has 19 cities
K-means for 2 has 1 cities
K-means for 3 has 16 cities
K-means for 4 has 9 cities
K-means for 5 has 27 cities
K-means for 6 has 4 cities
K-means for 7 has 1 cities


In [24]:
#show first most common venues per cluster

cluster0 = cities_merged.loc[cities_merged['Cluster Labels'] == 0, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]
cluster0['1st Most Common Venue'].value_counts().head()

Hotel          4
Pub            4
Coffee Shop    3
Supermarket    1
Bar            1
Name: 1st Most Common Venue, dtype: int64

In [25]:
cluster1 = cities_merged.loc[cities_merged['Cluster Labels'] == 1, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]
cluster1['1st Most Common Venue'].value_counts().head()

Tapas Restaurant    13
Plaza                2
Restaurant           2
Hotel                1
Bar                  1
Name: 1st Most Common Venue, dtype: int64

In [26]:
cluster2 = cities_merged.loc[cities_merged['Cluster Labels'] == 2, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]
cluster2 ['1st Most Common Venue'].value_counts().head()

Harbor / Marina    1
Name: 1st Most Common Venue, dtype: int64

In [27]:
cluster3 = cities_merged.loc[cities_merged['Cluster Labels'] == 3, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]
cluster3['1st Most Common Venue'].value_counts().head()

Hotel                       6
Tapas Restaurant            3
Mediterranean Restaurant    3
Spanish Restaurant          3
Plaza                       1
Name: 1st Most Common Venue, dtype: int64

In [28]:
cluster4 = cities_merged.loc[cities_merged['Cluster Labels'] == 4, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]
cluster4['1st Most Common Venue'].value_counts().head()

Spanish Restaurant    8
Historic Site         1
Name: 1st Most Common Venue, dtype: int64

In [29]:
cluster5 = cities_merged.loc[cities_merged['Cluster Labels'] == 5, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]
cluster5['1st Most Common Venue'].value_counts().head()

Pub                  17
Coffee Shop           4
Indian Restaurant     2
Bar                   2
Beach                 1
Name: 1st Most Common Venue, dtype: int64

In [30]:
cluster6 = cities_merged.loc[cities_merged['Cluster Labels'] == 6, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]
cluster6['1st Most Common Venue'].value_counts().head()

Coffee Shop    2
Pub            2
Name: 1st Most Common Venue, dtype: int64

In [31]:
cluster7 = cities_merged.loc[cities_merged['Cluster Labels'] == 7, cities_merged.columns[[1] + list(range(5, cities_merged.shape[1]))]]
cluster7['1st Most Common Venue'].value_counts().head()

Pub    1
Name: 1st Most Common Venue, dtype: int64

In [32]:
#show table with 10 most common venues per city, all grouped by cluster
grouped = cities_merged.groupby(['Cluster Labels', 'city'])
grouped.first().head()

lat   lng         country  population  \
Cluster Labels city                                                  
0              Aberdeen  57.1704 -2.08  United Kingdom      189364   
               Bath      51.3837 -2.35  United Kingdom       93238   
               Belfast   54.6000 -5.96  United Kingdom      450406   
               Carlisle  54.8800 -2.93  United Kingdom       72633   
               Chester   53.2000 -2.92  United Kingdom       89531   

                        1st Most Common Venue 2nd Most Common Venue  \
Cluster Labels city                                                   
0              Aberdeen                   Bar         Grocery Store   
               Bath                       Pub           Coffee Shop   
               Belfast             Restaurant                   Bar   
               Carlisle           Supermarket                   Bar   
               Chester            Zoo Exhibit                   Pub   

                        3rd Most Common Venue 4th Most Common Venue  \
Cluster Labels city                                                   
0              Aberdeen                  Café            Restaurant   
               Bath                      Park                 Hotel   
               Belfast                   Café                   Pub   
               Carlisle                  Café                   Pub   
               Chester            Supermarket                   Bar   

                        5th Most Common Venue 6th Most Common Venue  \
Cluster Labels city                                                   
0              Aberdeen                 Hotel                  Park   
               Bath             Grocery Store          Cocktail Bar   
               Belfast            Coffee Shop           Pizza Place   
               Carlisle                 Hotel         Grocery Store   
               Chester            Coffee Shop                 Hotel   

                        7th Most Common Venue 8th Most Common Venue  \
Cluster Labels city                                                   
0              Aberdeen           Supermarket              Beer Bar   
               Bath                Restaurant                  Farm   
               Belfast           Cocktail Bar                Bistro   
               Carlisle        Clothing Store           Coffee Shop   
               Chester                   Café         Grocery Store   

                        9th Most Common Venue 10th Most Common Venue  
Cluster Labels city                                                   
0              Aberdeen        Sandwich Place     Seafood Restaurant  
               Bath                   Theater        Bed & Breakfast  
               Belfast                  Hotel                   Park  
               Carlisle                Bakery   Gym / Fitness Center  
               Chester     Italian Restaurant            Pizza Place

In [33]:
#display countries for each cluster
pd.set_option('display.max_colwidth', 500)
cities_merged.groupby('Cluster Labels')['city'].apply(list)
df7 = cities_merged.groupby('Cluster Labels')['city'].apply(list).reset_index(name='list of cities')
df7

,Cluster Labels,list of cities
0,0,"[Glasgow, Liverpool, Edinburgh, Belfast, Aberdeen, Dundee, Exeter, Bath, Chester, Derry, Carlisle, Scarborough, Inverness, Perth, Dover, Fort William, Kirkwall]"
1,1,"[Madrid, Sevilla, Bilbao, Zaragoza, Vigo, Las Palmas de Gran Canaria, Donostia, Santa Cruz de Tenerife, Valladolid, Alicante, Vitoria-Gasteiz, Almería, Albacete, Logroño, Ciudad de Melilla, Badajoz, León, Ourense, Jaén]"
2,2,[Lerwick]
3,3,"[London, Barcelona, Valencia, Málaga, Murcia, Granada, Palma, Cartagena, Marbella, Mataró, Salamanca, Tarragona, Algeciras, Santiago de Compostela, Ciudad de Ceuta, Arrecife]"
4,4,"[Gijón, Córdoba, Pamplona, Oviedo, Castellón de la Plana, Burgos, Huelva, Toledo, Mérida]"
5,5,"[Birmingham, Manchester, Leeds, Sheffield, Newcastle upon Tyne, Caerdydd, Nottingham, Southend-on-Sea, Bristol, Brighton, Bradford, Leicester, Sunderland, Portsmouth, Bournemouth, Coventry, Southampton, Reading, Kingston upon Hull, Blackpool, Plymouth, Oxford, Norwich, York, Ipswich, Peterborough, Cambridge]"
6,6,"[Middlesbrough, Stoke-on-Trent, Luton, Dumfries]"
7,7,[Omagh]


In [34]:
#get specific venues per city
groupedvenues.drop(groupedvenues.columns.difference(['River','city']), 1, inplace=True)

In [35]:
#show specific venues per city and add Cluster Labels column
groupedvenues.insert(0, 'Cluster Labels', kmeans.labels_)
groupedvenues.head()

,Cluster Labels,city,River
0,0,Aberdeen,0.0
1,1,Albacete,0.0
2,3,Algeciras,0.0
3,1,Alicante,0.0
4,1,Almería,0.0


In [36]:
#drop rows with 0 values
groupedvenues[(groupedvenues != 0).all(1)]

,Cluster Labels,city,River
76,3,Salamanca,0.013333


In [37]:
#create new dataframe to merge filtered groupedvenues dataframe with initial dataframe to get coordinates

df2 = pd.merge(groupedvenues, df, on=["city"])
df3=df2[(df2 != 0).all(1)]

In [38]:
# create map
map_clusters = folium.Map()

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df3['lat'], df3['lng'], df3['city'], df3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters